# Exploring NASA Planetary Mission Budget in SQL and Plotly express 🚀


For much of the last 60 years, NASA has been at the forefront of exploring our solar system. In this live training, we'll see how much money they spent to do this.

For expensive science projects, not least those funded with public money, the price of conducting research is a huge consideration, so budgeting is important.

Here, we'll use a cleaned up version of the [public dataset](https://www.planetary.org/space-policy/planetary-exploration-budget-dataset) provided by the The Planetary Society.

## What is the total cost of all planetary missions over all time?

In [95]:
-- Total cost 
SELECT ROUND(SUM("cost_MUSD")::numeric, 1)as total_cost_MUSD
FROM mission_budgets;

,total_cost_musd
0,41406.6


In [96]:
-- Cumulative cost 
WITH cte AS (SELECT fiscal_year as year, 
      SUM("cost_MUSD") as cost
      FROM mission_budgets
      GROUP BY year
      ORDER BY year ASC)
SELECT year, 
		SUM(COST) OVER (ORDER BY year ASC) as cumulative_total_musd
FROM cte
ORDER BY year ASC;

,year,cumulative_total_musd
0,1960,19.542000
1,1961,71.662000
2,1962,202.143000
3,1963,390.718000
4,1964,600.338000
...,...,...
72,2031,40851.270567
73,2032,41000.861294
74,2033,41150.454506
75,2034,41278.504506


In [97]:
# cumulative total over the years

import plotly.express as px

px.bar(
       cumul_df,
       x="year",
       y='cumulative_total_musd',
       title = 'Cumulative Budget of NASA Planetary Exploration',
       template="simple_white"
)

Due to inflation, costs in the 1960s are not directly equivalent to those in the 2020s. We need to adjust for inflation in order to get a cost in current currency.

Correction factors are available in the `inflation` table. Each row represents an inflation adjustment for a time period relative to today. ther are two columns.

In [98]:
-- Total cost of all missions over all years, adjusted for inflation


SELECT ROUND(SUM("cost_MUSD")::numeric, 1) as total_cost_MUSD,
		ROUND(SUM("cost_MUSD" * inflation_adjustment)::numeric,1) as inflation_adjusted_total_cost_MUSD
FROM mission_budgets
LEFT JOIN inflation
USING(fiscal_year)
;


,total_cost_musd,inflation_adjusted_total_cost_musd
0,41406.6,80992.6


**Since 1960, NASA spent over 41 billion US dollar for planetary exploration. However, when we take inflation into account, the total cost represents around 81 billion USD.**



## Which was the most expensive mission?

The biggest, grandest missions make headlines, but at some point, someone always asks questions about how much things cost, and the biggest budgets are the first place people look for cost savings. Knowing which is the most expensive project is an essential task for anyone responsible for a budget.

In [99]:
-- Mission with the highest total cost

SELECT mission, 
		--ROUND(SUM("cost_MUSD")::numeric, 1) as not_adjusted_total_cost_MUSD,
		ROUND(SUM("cost_MUSD" * inflation_adjustment)::numeric,1) as inflation_adjusted_total_cost_MUSD
FROM mission_budgets
LEFT JOIN inflation
USING(fiscal_year)
GROUP BY mission
ORDER BY inflation_adjusted_total_cost_MUSD DESC
LIMIT 3
;


,mission,inflation_adjusted_total_cost_musd
0,Viking,7208.5
1,Surveyor 1 - 7,6333.9
2,Cassini,5285.4


**The most expensive mission was Viking, costing around 7.2 billion USD.**

## How much was spent each year?

Understanding how budgets change over time is also important. NASA has not been immune to the wax and wane of the US economy over the decades, nor the change in political enthusiasm for planetary exploration. More generally, to understand your current budget, it's usually helpful to know the context of how much budget you or your predecessors had in previous years.

In [100]:
-- Inflation adjusted total cost per year

SELECT fiscal_year as year, 
		ROUND(SUM("cost_MUSD" * inflation_adjustment)::numeric,1) as adjusted_total_cost_MUSD
FROM mission_budgets
LEFT JOIN inflation
	USING(fiscal_year)
GROUP BY year
ORDER BY year ASC
;

,year,adjusted_total_cost_musd
0,1960,261.1
1,1961,674.8
2,1962,1624.5
3,1963,2268.4
4,1964,2412.9
...,...,...
72,2031,149.6
73,2032,149.6
74,2033,149.6
75,2034,128.1


In [101]:
# cost vs. year as line

px.line(
       total_cost_by_year,
       x="year",
       y='adjusted_total_cost_musd',
      title = 'NASA Planetary Exploration budget over time',
       template="simple_white"
)



**We observe a couple of peaks around the mid 60s and 70s followed by a period where budget was lower  but gradually increasing up to today.**

## How much was spent on each destination?

The 1960s was famous for the "Space Race" between the United States and Russia to be the first to get a man on the moon. In the time since then, attention has also been turned to the other planets in the solar system and to small bodies like asteroids and comets.

In [102]:
-- Inflation adjusted total cost per destination
-- Destination with the highest total cost

SELECT destination,
		ROUND(SUM("cost_MUSD" * inflation_adjustment)::numeric,1) as inflation_adjusted_total_cost_MUSD
FROM mission_budgets
LEFT JOIN inflation
	USING(fiscal_year)
LEFT JOIN mission_details
	USING(mission)
GROUP BY destination
ORDER BY inflation_adjusted_total_cost_MUSD DESC
;

,destination,inflation_adjusted_total_cost_musd
0,Mars,31725.3
1,Outer Planets,21827.6
2,The Moon,12982.9
3,Small Bodies,7365.9
4,Venus,4796.7
5,Mercury,1519.6
6,Earth-Sun L1,427.2
7,The Sun,347.4


In [103]:
# This is a chart, switch to the DataCamp editor to view and configure it.

,destination,inflation_adjusted_total_cost_musd
0,Mars,31725.3
1,Outer Planets,21827.6
2,The Moon,12982.9
3,Small Bodies,7365.9
4,Venus,4796.7
5,Mercury,1519.6
6,Earth-Sun L1,427.2
7,The Sun,347.4


**Mars is the most expensive destination, followed by the Outer Planets and the Moon.**

## How did spending by destination change over time?

In [104]:
-- Cost of each destination over time

SELECT destination, 
		fiscal_year as year,
		ROUND(SUM("cost_MUSD" * inflation_adjustment)::numeric,1) as inflation_adjusted_total_cost_MUSD
FROM mission_budgets
LEFT JOIN inflation
	USING(fiscal_year)
LEFT JOIN mission_details
	USING(mission)
GROUP BY destination, year
ORDER BY year ASC
;

,destination,year,inflation_adjusted_total_cost_musd
0,The Moon,1960,261.1
1,The Moon,1961,674.8
2,The Moon,1962,1248.2
3,Venus,1962,376.3
4,Outer Planets,1963,31.4
...,...,...,...
277,Outer Planets,2032,128.1
278,Outer Planets,2033,128.1
279,Small Bodies,2033,21.5
280,Outer Planets,2034,128.1


In [105]:
# total cost vs. year, colored by destination
fig = px.bar(
    total_cost_by_date_and_destination,
    x="year",
    y="inflation_adjusted_total_cost_musd",
    color="destination",
    template="simple_white",
      title = 'NASA Planetary Exploration budget per destinations over time'
)
fig.update_layout(xaxis={'categoryorder': 'category ascending'})
fig.show()

**In the 60s, the majority of the budget was dedicated to the Moon missions, then in the 70s to the Mars missions.**

**In the 80s and 90s, much less budget was spent overall and was mainly dedicated to the the Outer planets missions with a short-lived investment in the Venus missions.**

**In the last 20 years, most of the budget has been focused on Mars missions again and Outer planets.**


## Which part of the mission is most expensive?

In [106]:
-- average cost broken down per cost type (regardless of mission)

SELECT cost_type, 
       ROUND(AVG("cost_MUSD" * inflation_adjustment)::numeric,2) as inflation_adjusted_total_cost_MUSD
FROM mission_budgets
LEFT JOIN inflation
	USING(fiscal_year)
GROUP BY cost_type
ORDER BY inflation_adjusted_total_cost_MUSD DESC

,cost_type,inflation_adjusted_total_cost_musd
0,Implementation (incl LV),207.10
1,Spacecraft,187.29
2,Development (incl LV),154.43
3,Science,118.83
4,Formulation,111.02
5,Launch Vehicle(s),95.52
6,Development,76.29
7,Implementation,67.33
8,Experiments,49.61
9,Launch Support,39.91


In [107]:
# This is a chart, switch to the DataCamp editor to view and configure it.

,cost_type,inflation_adjusted_total_cost_musd
0,Implementation (incl LV),207.10
1,Spacecraft,187.29
2,Development (incl LV),154.43
3,Science,118.83
4,Formulation,111.02
5,Launch Vehicle(s),95.52
6,Development,76.29
7,Implementation,67.33
8,Experiments,49.61
9,Launch Support,39.91


**The most expensive part of the mission is Implementation, followed by Spacecraft and Development.**



## Which programs have more than one destination for their missions?

In [108]:
-- number of missions for each program

SELECT program, COUNT(mission) as num_missions
FROM mission_details
GROUP BY program
HAVING COUNT(mission)>1
order by num_missions DESC;

,program,num_missions
0,Discovery,14
1,Mars Exploration,9
2,Mariner,6
3,,5
4,Pioneer,4
5,New Frontiers,3
6,SIMPLEx,3
7,Lunar Discovery and Exploration,2
8,Solar System Exploration,2


**Nine programs had more than 1 mission. The program that had the most mission was the Discovery program with a total of 14 missions, followed by the Mars Exploration program with 9 missions.**

# Conclusion

This concludes our exploratory analysis of the NASA Planetary Exploration Budgets. Here we saw how much was spent over the year and on each missions (not forgetting inflation!), we also looked at the cost breakdown for each mission across various elements and finally looked for the pragrams with the highest number of missions. 

I hope you found this informative and thanks for reading!